# AI21 Example

First let's install the required packages

In [ ]:
%pip install -U ai21
%pip install -U agentops

Then import them

In [ ]:
from ai21 import AI21Client, AsyncAI21Client
from ai21.models.chat import ChatMessage
from dotenv import load_dotenv
import os
import asyncio
import agentops

Next, we'll grab our API keys. You can use dotenv like below or however else you like to load environment variables

In [ ]:
load_dotenv()
AI21_API_KEY = os.getenv("AI2I_API_KEY") or "<your_ai21_key>"
AGENTOPS_API_KEY = os.getenv("AGENTOPS_API_KEY") or "<your_agentops_key>"

In [ ]:
agentops.init(AGENTOPS_API_KEY, default_tags=["ai21-example"])

# Setting up Messages
AI21 clients use a `ChatMessage` object to handle messages. We setup the following system prompt to guide the model in its response and a user prompt as well. We take the example of a support agent in a SaaS company.

In [ ]:
messages = [
    ChatMessage(
        content="You are a world renowned poet in the style of Edgar Allan Poe.",
        role="system",
    ),
    ChatMessage(
        content="Write me a short poem about the AI agents co-existing within the human brain.",
        role="user",
    ),
]

# Sync Examples

We will demonstrate a basic sync call to AI21 using the Jamba 1.5 model

In [ ]:
client = AI21Client(api_key=AI21_API_KEY)

In [ ]:
response = client.chat.completions.create(
    messages=messages,
    model="jamba-1.5-mini",
)
print(response.choices[0].message.content)

The following example shows how to record data from the streamed response using the Jamba 1.5 model.

In [ ]:
response = ""

stream_response = client.chat.completions.create(
    messages=messages,
    model="jamba-instruct",
    stream=True,
)

for chunk in stream_response:
    response += str(chunk.choices[0].delta.content)

print(response)

## Async Example
The async example is very similar to the sync example, but it uses the `AsyncAI21Client` class.

In [ ]:
aclient = AsyncAI21Client(api_key=AI21_API_KEY)

In [ ]:
async def main():
    async_response = await aclient.chat.completions.create(
        messages=messages,
        model="jamba-1.5-mini",
    )
    print(async_response.choices[0].message.content)


await main()

The following example shows how to record data from the async streamed response.

In [ ]:
async def main():
    response = ""

    async_stream_response = await aclient.chat.completions.create(
        messages=messages,
        model="jamba-1.5-mini",
        stream=True,
    )

    async for chunk in async_stream_response:
        response += chunk.choices[0].delta.content

    print(response)


await main()

# Task-Specific Models Examples

## Contextual Answers

The following example demonstrates the answering capability of AI21 without streaming.

In [ ]:
CONTEXT = """
In 2020 and 2021, enormous QE — approximately $4.4 trillion, or 18%, of 2021 gross
domestic product (GDP) — and enormous fiscal stimulus (which has been and
always will be inflationary) — approximately $5 trillion, or 21%, of 2021 GDP
— stabilized markets and allowed companies to raise enormous amounts of
capital. In addition, this infusion of capital saved many small businesses and
put more than $2.5 trillion in the hands of consumers and almost $1 trillion into
state and local coffers. These actions led to a rapid decline in unemployment, 
dropping from 15% to under 4% in 20 months — the magnitude and speed of which were both
unprecedented. Additionally, the economy grew 7% in 2021 despite the arrival of
the Delta and Omicron variants and the global supply chain shortages, which were
largely fueled by the dramatic upswing in consumer spending and the shift in
that spend from services to goods.
"""
response = client.answer.create(
    context=CONTEXT,
    question="Did the economy shrink after the Omicron variant arrived?",
)
print(response.answer)

Similarly, we can use streaming to get the answer.

In [ ]:
CONTEXT = """
In the rapidly evolving field of Artificial Intelligence (AI), mathematical 
foundations such as calculus, linear algebra, and statistics play a crucial role. 
For instance, linear algebra is essential for understanding and developing machine 
learning algorithms. It involves the study of vectors, matrices, and tensor operations 
which are critical for performing transformations and optimizations. Additionally, 
concepts from calculus like derivatives and integrals are used to optimize the 
performance of AI models through gradient descent and other optimization techniques. 
Statistics and probability form the backbone for making inferences and predictions, 
enabling AI systems to learn from data and make decisions under uncertainty. 
Understanding these mathematical principles allows for the development of more robust 
and effective AI systems.
"""
response = client.answer.create(
    context=CONTEXT,
    question="Why is linear algebra important for machine learning algorithms?",
    stream=True,
)
print(response.answer)

In [ ]:
agentops.end_session("Success")